In [ ]:
# import modules
import math
import sys
import pandas as pd
from matplotlib import pyplot as plt
import networkx as nx

from leap_ec.individual import Individual
from leap_ec.decoder import IdentityDecoder
from leap_ec.representation import Representation
from leap_ec.algorithm import multi_population_ea
from leap_ec.context import context

import leap_ec.ops as ops
from leap_ec import probe
from leap_ec.algorithm import multi_population_ea

from leap_ec.real_rep.problems import SchwefelProblem
from leap_ec.real_rep.ops import mutate_gaussian
from leap_ec.real_rep.initializers import create_real_vector

import yfinance as yf

# Read Data

In [ ]:
df = yf.download("AMZN",period = "max", interval = "1d")
df['Range'] = df['High'] - df['Low']
df['RangeOC'] = df['Open'] - df['Adj Close']
df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Range,RangeOC
Date,,,,,,,,
1997-05-15,2.437500,2.500000,1.927083,1.958333,1.958333,72156000,0.572917,0.479167
1997-05-16,1.968750,1.979167,1.708333,1.729167,1.729167,14700000,0.270834,0.239583
1997-05-19,1.760417,1.770833,1.625000,1.708333,1.708333,6106800,0.145833,0.052084
1997-05-20,1.729167,1.750000,1.635417,1.635417,1.635417,5467200,0.114583,0.093750
1997-05-21,1.635417,1.645833,1.375000,1.427083,1.427083,18853200,0.270833,0.208334


# Identify trend day and features day

In [ ]:
# Up and Down trend day
df.loc[(df['Open'] <= df['Low']+0.1*df['Range']) & (df['Adj Close'] >= df['High']-0.2*df['Range']), 'Up_Trend'] = 1
df.loc[(df['Open'] >= df['High']-0.1*df['Range']) & (df['Adj Close'] <= df['High']+0.2*df['Range']), 'Down_Trend'] = 1
df['Up_Trend'] = df['Up_Trend'].fillna(0)
df['Down_Trend'] = df['Down_Trend'].fillna(0)

In [ ]:
# NRk
for i in range(2,8):
    df['Previous{k}'.format(k = i)] = df['Range'].rolling(i).min().shift(i-1)
    df.loc[df['Range'] <= df['Previous{k}'.format(k = i)], 'NR{k}'.format(k = i)] = 1
    df = df.drop(['Previous{k}'.format(k = i)], axis=1)
    df['NR{k}'.format(k = i)] = df['NR{k}'.format(k = i)].fillna(0)

In [ ]:
# DOJI
# TODO: What does x effect? x: a small percentage of the range
x = 0.05
df.loc[df['RangeOC'].abs() <= x*df['Range'], 'DOJI'] = 1
df['DOJI'] = df['DOJI'].fillna(0)

In [ ]:
# Hook day
for delta in range(0, 11):
    df.loc[df['Open'].shift(-1) < df['Low']-delta, 'Up_Hook{d}'.format(d=delta)] = 1
    df.loc[df['Open'].shift(-1) > df['High']+delta, 'Down_Hook{d}'.format(d=delta)] = 1
    df['Up_Hook{d}'.format(d=delta)] = df['Up_Hook{d}'.format(d=delta)].fillna(0)
    df['Down_Hook{d}'.format(d=delta)] = df['Down_Hook{d}'.format(d=delta)].fillna(0)

# Features day membership functions

In [ ]:
# NRk membership function
ks = [4,6,7]
cs = [1/2,1/3,1/3]
v_mins = [2,3,4]
v_maxs = [4,6,7]

for i in range(0,len(ks)):
    # Generate other parameters in the function
    df['D{k}'.format(k = ks[i])] = df['NR{k}'.format(k = ks[i])].shift(1).rolling(ks[i]).sum()
    df['~R{k}'.format(k = ks[i])] = df['Range'].shift(1).rolling(ks[i]).max()
    df['eta{k}'.format(k = ks[i])] = (df['~R{k}'.format(k = ks[i])]-df['Range'])/df['~R{k}'.format(k = ks[i])]
    df['NR_x{k}'.format(k = ks[i])] = df['D{k}'.format(k = ks[i])] + df['eta{k}'.format(k = ks[i])]
    # Conditioinal membership function
    df.loc[df['NR_x{k}'.format(k = ks[i])] < v_mins[i], 'NR{k}_membership'.format(k = ks[i])] = 0
    df.loc[(df['NR_x{k}'.format(k = ks[i])] >= v_mins[i]) & (df['NR_x{k}'.format(k = ks[i])] < v_maxs[i]), 
           'NR{k}_membership'.format(k = ks[i])] = cs[i]*(df['NR_x{k}'.format(k = ks[i])] - v_mins[i])
    df.loc[df['NR_x{k}'.format(k = ks[i])] >= v_maxs[i], 'NR{k}_membership'.format(k = ks[i])] = 1
    #Drop unecessary columns
    df = df.drop(['D{k}'.format(k = ks[i])], axis=1)
    df = df.drop(['~R{k}'.format(k = ks[i])], axis=1)
    df = df.drop(['eta{k}'.format(k = ks[i])], axis=1)
    df = df.drop(['NR_x{k}'.format(k = ks[i])], axis=1)
    
#df[['Range','NR4','D4','~R4','eta4','NR_x4']].tail(10)
#df[df['NR4_membership'] != 0]

In [ ]:
# DOJI membership function
rhos = [0.05,0.10,0.15,0.20,0.25,0.30]
for rho in rhos:
    df.loc[(df['RangeOC'] >= 0) & (df['RangeOC']<= 2), 'DOJI{r}'.format(r = rho) ] = 1 - (df['RangeOC']/rho)
    df['DOJI{r}_membership'.format(r = rho)] = df['DOJI{r}'.format(r = rho)].fillna(0)

In [ ]:
# Hook day membership function
for delta in range(0, 11):
    # Generate other parameters in the function
    df.loc[(df['Up_Hook{d}'.format(d=delta)] == 1), 'Hook{d}_x'.format(d=delta)] = df['Low']-delta-df['Open'].shift(1)
    df.loc[(df['Down_Hook{d}'.format(d=delta)] == 1), 'Hook{d}_x'.format(d=delta)] = df['Open'].shift(1)-delta-df['High']
    # Conditioinal membership function
    df.loc[(df['Hook{d}_x'.format(d=delta)] < -1/2), 'Hook{d}_membership'.format(d=delta)] = 0
    df.loc[(df['Hook{d}_x'.format(d=delta)] >= -1/2) & (df['Hook{d}_x'.format(d=delta)] < 0), 
           'Hook{d}_membership'.format(d=delta)] = 2*(df['Hook{d}_x'.format(d=delta)] + 0.5)
    df.loc[(df['Hook{d}_x'.format(d=delta)] >= 0), 'Hook{d}_membership'.format(d=delta)] = 1
    
#df[(df['Hook2_membership']>0)&(df['Hook2_membership']!=1)]

In [ ]:
df

,Open,High,Low,Close,Adj Close,Volume,Range,RangeOC,Up_Trend,Down_Trend,...,Hook6_x,Hook6_membership,Hook7_x,Hook7_membership,Hook8_x,Hook8_membership,Hook9_x,Hook9_membership,Hook10_x,Hook10_membership
Date,,,,,,,,,,,,,,,,,,,,,
1997-05-15,2.437500,2.500000,1.927083,1.958333,1.958333,72156000,0.572917,0.479167,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-05-16,1.968750,1.979167,1.708333,1.729167,1.729167,14700000,0.270834,0.239583,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-05-19,1.760417,1.770833,1.625000,1.708333,1.708333,6106800,0.145833,0.052084,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-05-20,1.729167,1.750000,1.635417,1.635417,1.635417,5467200,0.114583,0.093750,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-05-21,1.635417,1.645833,1.375000,1.427083,1.427083,18853200,0.270833,0.208334,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-08,3015.000000,3064.590088,2951.310059,2951.949951,2951.949951,4178500,113.280029,63.050049,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-09,3017.989990,3090.959961,3005.149902,3062.850098,3062.850098,4023500,85.810059,-44.860107,0.0,0.0,...,-81.959961,0.0,-82.959961,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-10,3098.449951,3116.459961,3030.050049,3057.639893,3057.639893,3008700,86.409912,40.810059,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Results of features day memberships

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9433f6a9-5256-43e6-8a88-84eded61d4fc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>